In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

### Carregando os dados limpos e normalizados

In [3]:
# Carregando os dados
data = pd.read_csv('../data/processed/cardiovascular_processed_scaled.csv', sep=';')
X = data.drop('target', axis=1)
y = data['target']

scaler = joblib.load('../data/processed/scaler.pkl')

param_grid = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100, 1000, 5000, 10000],
            'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg', 'powell', 'trust-constr'],
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'max_iter': [100, 200, 300, 400, 500, 1000, 2000],
            'class_weight': [None, 'balanced']
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4, 8],
            'bootstrap': [True, False]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100, 1000, 5000, 10000],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid', 'precomputed', 'cosine']
        }
    },
    'XGBoost': {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
            'max_depth': [3, 5, 7, 9, 11, 13],
            'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
        }
    }
}

In [8]:
results = {}

for model_name, config in param_grid.items():
    print(f'\nTreinando {model_name}...')

    pipe = Pipeline([
        ('scaler', scaler),
        ('model', config['model'])
    ])

    # Prefixo model__ é OBRIGATÓRIO
    params = {
        f'model__{k}': v for k, v in config['params'].items()
    }

    scorings = {
        'acuracia': 'accuracy',
        'recall': 'recall',
        'f1_score': 'f1',
        'precision': 'precision'
    }

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=params,
        cv=5,
        scoring=scorings,
        refit='recall',
        n_jobs=-1,
        verbose=1
    )

    grid.fit(X, y)

    results[model_name] = grid


    print(f'Best Recall: {grid.best_score_:.4f}')


Treinando Logistic Regression...
Fitting 5 folds for each of 2688 candidates, totalling 13440 fits


C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
10080 fits failed out of a total of 13440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
560 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\Lo

Best Recall: 0.9753

Treinando Random Forest...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best Recall: 0.9886

Treinando SVC...
Fitting 5 folds for each of 42 candidates, totalling 210 fits


C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
70 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Guilherme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

Best Recall: 0.9753

Treinando XGBoost...
Fitting 5 folds for each of 5400 candidates, totalling 27000 fits
Best Recall: 0.9905


In [13]:
for model_name, grid in results.items():
    print(f'\n===== {model_name} =====')

    idx = grid.best_index_

    print('Melhores parâmetros:')
    for k, v in grid.best_params_.items():
        print(f'  {k}: {v}')

    print('\nScores médios (CV):')
    print(f"  Accuracy : {grid.cv_results_['mean_test_acuracia'][idx]:.4f}")
    print(f"  Recall   : {grid.cv_results_['mean_test_recall'][idx]:.4f}")
    print(f"  Precision: {grid.cv_results_['mean_test_precision'][idx]:.4f}")
    print(f"  F1-score : {grid.cv_results_['mean_test_f1_score'][idx]:.4f}")



===== Logistic Regression =====
Melhores parâmetros:
  model__C: 1
  model__class_weight: None
  model__max_iter: 100
  model__penalty: l1
  model__solver: saga

Scores médios (CV):
  Accuracy : 0.9662
  Recall   : 0.9753
  Precision: 0.9647
  F1-score : 0.9698

===== Random Forest =====
Melhores parâmetros:
  model__bootstrap: False
  model__max_depth: None
  model__min_samples_leaf: 1
  model__min_samples_split: 2
  model__n_estimators: 50

Scores médios (CV):
  Accuracy : 0.9768
  Recall   : 0.9886
  Precision: 0.9707
  F1-score : 0.9794

===== SVC =====
Melhores parâmetros:
  model__C: 0.1
  model__kernel: poly

Scores médios (CV):
  Accuracy : 0.9218
  Recall   : 0.9753
  Precision: 0.8943
  F1-score : 0.9329

===== XGBoost =====
Melhores parâmetros:
  model__colsample_bytree: 0.6
  model__learning_rate: 0.01
  model__max_depth: 7
  model__n_estimators: 50
  model__subsample: 0.9

Scores médios (CV):
  Accuracy : 0.9683
  Recall   : 0.9905
  Precision: 0.9549
  F1-score : 0.9722


In [14]:
# Selecionar o melhor modelo baseado no recall
best_model_name = max(results, key=lambda x: results[x].best_score_)
best_model = results[best_model_name].best_estimator_

print(f"Melhor modelo: {best_model_name}")
print(f"Melhor score (Recall): {results[best_model_name].best_score_:.4f}")

# Salvar o melhor modelo
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)
model_path = models_dir / 'best_model.pkl'
joblib.dump(best_model, model_path)
print(f"Modelo salvo em: {model_path}")

# Salvar o scaler também, se necessário (já foi salvo no feature engineering)
scaler_path = Path('../data/processed/scaler.pkl')
if not scaler_path.exists():
    joblib.dump(scaler, scaler_path)
    print(f"Scaler salvo em: {scaler_path}")

Melhor modelo: XGBoost
Melhor score (Recall): 0.9905
Modelo salvo em: ..\models\best_model.pkl
